# Multiple Loan Calculator

Ziyu Tang, Xinlan Wu

In [1]:
# packages for testing
import pandas as pd
import ipytest
ipytest.autoconfig()
import pytest

# packages for algorithm
from algorithm.LoanImpacts import *
from algorithm.LoanPortfolio import *
from algorithm.Loan import *
import plotly.graph_objects as go

## Create Functions

### Loan Output

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import decimal


class Helper:
    """ Helper class for printing and plotting of loan schedules.
    """
    @staticmethod
    def display(value, digits=2):
        """ Return a displayable value with a specified number of digits.
        :param value: value to display
        :param digits: number of digits right of the decimal place
        :return: formatted displayable value
        """
        temp = str(decimal.Decimal(str(value) + '0' * digits))
        return temp[:temp.find('.') + digits + 1]

    @staticmethod
    def plot(df):
        large_rockwell_template = dict(
                                       layout=go.Layout(title_font=dict(family="Rockwell", size=18))
                                      )
        
        fig = go.Figure()
        fig.add_trace(go.Bar(
                              x=df['Payment Date'],
                              y=df['Applied Principal'].tolist(),
                              name='Applied Principal',
                              marker_color='lightseagreen'
                              )                     
                     )
        fig.add_trace(go.Bar(
                              x=df['Payment Date'],
                              y=df['Applied Interest'].tolist(),
                              name='Applied Interest',
                              marker_color='navajowhite'
                              )                     
                     )
        
        
        fig.update_layout(title="Distribution of Applied Interest and Principal",template=large_rockwell_template,barmode='stack', xaxis_tickangle=-45)
        
        return fig


    @staticmethod
    def print(loan,date):
        df = pd.DataFrame(columns = ['Payment Date', 'Begin Principal', 'Payment', 'Extra Payment',
                         'Applied Principal', 'Applied Interest', 'Balance']) 
        for pay in loan.schedule.values():
            to_append = list(pay) 
            a_series = pd.Series(to_append, index = df.columns)
            df = df.append(a_series, ignore_index=True)
        
        def format(x):
            return "${:.2f}".format(x)
        
        for i in df.columns.tolist()[1:]:
            df[i] = df[i].apply(format)
        
        start = pd.to_datetime(date)
        month_range = start + pd.to_timedelta(np.arange(df.shape[0]), 'M')
        month_list = []
        for i in month_range:
            month_list.append(i.strftime('%b-%y'))
        
        df['Payment Date'] = month_list
        
        #df = df.style.hide_index()
        
        return df

### Contribution Output

In [3]:
#from Loan import Loan
#from LoanPortfolio import LoanPortfolio


class LoanImpacts:
    """ Contributor Impacts to Loan class
    """

    def __init__(self, principal, rate, payment, extra_payment, contributions):
        self.principal = principal
        self.rate = rate
        self.payment = payment
        self.extra_payment = extra_payment
        self.contributions = contributions

    def compute_impacts(self):
        df = pd.DataFrame(columns = ['Index', 'InterestPaid', 'Duration', 'MIInterest','MIDuration']) 
        index_list = ['All','No Contribution']
        interest_list = []
        duration_list = []
        mi_interest_list = []
        mi_duration_list = []
        
        # loan with all contributions (mi)_all
        #
        loan_all = Loan(principal=self.principal, rate=self.rate,
                        payment=self.payment, extra_payment=self.extra_payment + sum(self.contributions))
        loan_all.check_loan_parameters()
        loan_all.compute_schedule()
        
        interest_list.append(round(loan_all.total_interest_paid, 2))
        duration_list.append(loan_all.time_to_loan_termination)
        mi_interest_list.append("")
        mi_duration_list.append("")

        # loan with no contributions (mi)_0
        #
        loan_none = Loan(principal=self.principal, rate=self.rate,
                         payment=self.payment, extra_payment=self.extra_payment)
        loan_none.check_loan_parameters()
        loan_none.compute_schedule()

        micro_impact_interest_paid_all = \
            (loan_none.total_interest_paid - loan_all.total_interest_paid) / loan_all.total_interest_paid
        micro_impact_duration_all = -\
            (loan_none.time_to_loan_termination - loan_all.time_to_loan_termination) / loan_all.time_to_loan_termination

       
        interest_list.append(round(loan_none.total_interest_paid, 2))
        duration_list.append(loan_none.time_to_loan_termination)
        mi_interest_list.append(round(micro_impact_interest_paid_all, 4))
        mi_duration_list.append(round(micro_impact_duration_all, 4))

        
        # iterate over each contribution (mi)_index
        #
        for index, contribution in enumerate(self.contributions):
            loan_index = Loan(principal=self.principal, rate=self.rate, payment=self.payment,
                              extra_payment=self.extra_payment + sum(self.contributions) - contribution)
            loan_index.check_loan_parameters()
            loan_index.compute_schedule()

            micro_impact_interest_paid = \
                (loan_index.total_interest_paid - loan_all.total_interest_paid) / loan_all.total_interest_paid
            micro_impact_duration = \
                (loan_index.time_to_loan_termination - loan_all.time_to_loan_termination) / loan_all.time_to_loan_termination
            index_list.append(index+1)
            interest_list.append(round(loan_index.total_interest_paid, 2))
            duration_list.append(loan_index.time_to_loan_termination)
            mi_interest_list.append(round(micro_impact_interest_paid, 4))
            mi_duration_list.append(round(micro_impact_duration, 4))

        df['Index'] = index_list
        df['InterestPaid'] = interest_list
        df['Duration'] = duration_list
        df['MIInterest'] = mi_interest_list
        df['MIDuration'] = mi_duration_list

        return df

## Execute tests

In [4]:
%%run_pytest[clean]

loans = LoanPortfolio()

@pytest.mark.parametrize('principal, rate, payment, extra_payment',
                         [
                             (5000.0, 6.0, 96.66, 0.0),
                             (10000.0, 8.0, 121.33, 0.0),
                             (7000.0, 7.0, 167.62, 0.0),
                         ])
def test_loan(principal, rate, payment, extra_payment):
    loan = None
    try:
        loan = Loan(principal=principal, rate=rate, payment=payment, extra_payment=extra_payment)
        loan.check_loan_parameters()
        loan.compute_schedule()
    except ValueError as ex:
        print(ex)
    loans.add_loan(loan)
    df = Helper.print(loan,'2020-11-20')
    Helper.plot(df)
    Helper.print(loan,'2020-11-20')

    print(round(loan.total_principal_paid, 2), round(loan.total_interest_paid, 2),
          round(loan.time_to_loan_termination, 0))

    if loans.get_loan_count() == 3:
        loans.aggregate()
        dff = Helper.print(loans,'2020-11-20')
        Helper.plot(dff)
        Helper.print(loans,'2020-11-20')

    assert True


@pytest.mark.parametrize('principal, rate, payment, extra_payment, ' +
                         'total_principal_paid, total_interest_paid, time_to_loan_termination',
                         [
                             (27000.0, 4.0, 150.0, 0.0, 27000.0, 14303.0, 22 * 12.0 + 11.0),
                             (27000.0, 4.0, 150.0, 25.0, 27000.0, 10975.0, 18 * 12.0 + 2.0)
                         ])
def test_loan_with_extra_payment(principal, rate, payment, extra_payment,
                                 total_principal_paid, total_interest_paid,
                                 time_to_loan_termination):
    tolerance_for_cash = 5.0
    tolerance_for_time = 1.0

    loan = None
    try:
        loan = Loan(principal=principal, rate=rate, payment=payment, extra_payment=extra_payment)
        loan.check_loan_parameters()
        loan.compute_schedule()
    except ValueError as ex:
        print(ex)
    loans.add_loan(loan)
    df = Helper.print(loan,'2020-11-20')
    Helper.plot(df)

    print(round(loan.total_principal_paid, 2), round(loan.total_interest_paid, 2),
          round(loan.time_to_loan_termination, 0))

    assert abs(loan.total_principal_paid - total_principal_paid) <= tolerance_for_cash
    assert abs(loan.total_interest_paid - total_interest_paid) <= tolerance_for_cash
    assert abs(loan.time_to_loan_termination - time_to_loan_termination) <= tolerance_for_time

    if loans.get_loan_count() == 2:
        loans.aggregate()
        dff = Helper.print(loans,'2020-11-20')
        Helper.plot(loans)


@pytest.mark.parametrize('principal, rate, payment, extra_payment, contributions',
                         [
                             (68000.0, 4.0, 899.0, 0, [10, 100, 1000])
                         ])
def test_loan_contribution_1(principal, rate, payment, extra_payment, contributions):
    loan_impacts = LoanImpacts(principal=principal, rate=rate, payment=payment,
                               extra_payment=extra_payment, contributions=contributions)
    loan_impacts.compute_impacts()

    assert True


......                                                                   [100%]
6 passed in 1.53s


## App

In [5]:
import pandas as pd
from pandas import DataFrame
import numpy as np


import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import plotly.graph_objects as go
from dash.dependencies import Input, Output, State

from datetime import date
from datetime import datetime

from itertools import combinations

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

In [6]:
#============================================== Layout ==============================================#

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.LITERA])

# Loan information

loan_all = [[0,0,0,0,0],[0,0,0,0,0],[0,0,0,0,0]]

controls_self = dbc.Card(
    
    [
        html.Br(),
        dbc.FormGroup(
            [
                dbc.Label("Loan amount ($)"),
                dbc.Input(id="principal-input", type="number", value=""),
                dbc.FormText("Loan amount must be greater than 0.0"),
                dbc.FormFeedback(valid=True),
                dbc.FormFeedback(
                    "Invalid Loan Amount",
                    valid=False,
                ),
            ]
        ),
        
        dbc.FormGroup(
            [
                dbc.Label("Monthly payment ($)"),
                dbc.Input(id="min-payment-input", type="number", value=""),
                dbc.FormText("Monthly payment must be greater than 0.0"),
                dbc.FormFeedback(valid=True),
                dbc.FormFeedback(
                    "Invalid Monthly Payment",
                    valid=False,
                ),
            ]
        ),
        
        dbc.FormGroup(
            [
                dbc.Label("Interest rate per year (%)"),
                dbc.Input(id="interest-input", type="number", value=""),
                dbc.FormText("Interest rate must be greater than 0.0"),
                dbc.FormFeedback(valid=True),
                dbc.FormFeedback(
                    "Invalid Interest Rate",
                    valid=False,
                ),
            ]
        ),
        
        dbc.FormGroup(
            [
                dbc.Label("Monthly Extra Payment ($)"),
                dbc.Input(id="extra-input", type="number", value=0),
                dbc.FormText("Monthly extra payment must be no less than 0.0"),
                dbc.FormFeedback(valid=True),
                dbc.FormFeedback(
                    "Invalid Monthly Extra Payment",
                    valid=False,
                ),
            ]
        ),
         
        html.Br(),
        
        dbc.FormGroup(
            [
                dbc.Label("Start Date"),
                dcc.DatePickerSingle(
                                     id='date-picker-single',
                                     date = date.today()
                                     )
            ]
        ),      
        
        html.Br(),
        dbc.Button("Submit", outline=False, color="success", className="mr-1",id='submit-self-state',n_clicks = 0),
    ],
    body=True,
)


# Contributor information

controls_contributor = dbc.Card(dbc.CardBody(
    [
         dbc.Form(
    [
        dbc.FormGroup(
            [
                dbc.Label("Apply Loan #", className="mr-2"),
                dbc.Input(id="contribution_loan_number",type="number", placeholder="1,2,or 3"),
            ],
            className="mr-3",
        )
             
    ],
    inline=True,
),
        

    html.Br(),
        
        dbc.Form(
    [
        dbc.FormGroup(
            [
                dbc.Label("Contributor 1", className="mr-2"),
                dbc.Input(id="contribution1_name",type="text", placeholder="Name"),
            ],
            className="mr-3",
        ),
        dbc.FormGroup(
            [
                dbc.Label("Contribution Amount", className="mr-2"),
                dbc.Input(id="contribution1_amount",type="number", placeholder="No less than 0.0"),
                dbc.FormFeedback(valid=True),
                dbc.FormFeedback(
                    "Invalid Contribution Amount",
                    valid=False,
                )
            ],
            className="mr-3",       
        ),
             
    ],
    inline=True,
),
        

    html.Br(),
    
    dbc.Form(
    [
        dbc.FormGroup(
            [
                dbc.Label("Contributor 2", className="mr-2"),
                dbc.Input(id="contribution2_name",type="text", placeholder="Name"),
            ],
            className="mr-3",
        ),
        dbc.FormGroup(
            [
                dbc.Label("Contribution Amount", className="mr-2"),
                dbc.Input(id="contribution2_amount",type="number", placeholder="No less than 0.0"),
                dbc.FormFeedback(valid=True),
                dbc.FormFeedback(
                    "Invalid Contribution Amount",
                    valid=False,
                )
            ],
            className="mr-3",       
        ),
             
    ],
    inline=True,
),
        

    html.Br(),
    
    dbc.Form(
    [
        dbc.FormGroup(
            [
                dbc.Label("Contributor 3", className="mr-2"),
                dbc.Input(id="contribution3_name",type="text", placeholder="Name"),
            ],
            className="mr-3",
        ),
    
    dbc.FormGroup(
            [
                dbc.Label("Contribution Amount", className="mr-2"),
                dbc.Input(id="contribution3_amount",type="number", placeholder="No less than 0.0"),
                dbc.FormFeedback(valid=True),
                dbc.FormFeedback(
                    "Invalid Contribution Amount",
                    valid=False,
                )
            ],
            className="mr-3",       
        ),
    
             
    ],
    inline=True,
),
    html.Br(),
    
    dbc.Button("Apply Contribution Impact", outline=False, color="success", className="mr-1",id='submit-contributor-state',n_clicks = 0),
    ]
))



# Application layout

app.layout = dbc.Container(
    [
        dbc.Row(
            dbc.Col(
                html.H1("Loan Calculator")
            )
        ),
        
        dbc.Row(
            dbc.Col(
                html.P("This loan calculator will help you determine the monthly payments on a loan. \
                Simply enter the loan amount, monthly payment, interest rate, extra payment, and start date \
                in the fields below and click submit. You are able to have 3 loans in total as your loan portfolio. \
                You can also know the contribution impact by entering the payment from each contributor."), md=8
            )  
        ),
        
        dbc.Row(
            [
                dcc.Store(id='memory_input'),
                dcc.Store(id='memory_contributor'),
                dbc.Col(controls_self, md=4),
                dbc.Col([(controls_contributor),
                dbc.Card(dbc.CardBody([
                                                html.P("Monthly Payments",
                                                        style={
                                                                 'textAlign': 'center'
                                                                 }
                                                      ),
                                                                
                                                html.H3(html.Div(id='monthly payment overview'),
                                                       style={
                                                                 'textAlign': 'center',
                                                                 'fontSize': 40
                                                                 }),
                                                html.Br(),                    
                                                html.P(html.Div(id='Total Principal Paid')),
                                                html.Hr(),
                                                html.P(html.Div(id='Total Interest Paid')),
                                                html.Br(),      
                                                dbc.Button("Show/Hide Amortization Schedule", id="fade-button", color="info", className="mb-3"),
                                             ]
                                             ))], md=8),
            ],
            align="top",
        ),
   
        html.Br(),
        
        dbc.Col(
            dbc.Fade(
                     html.Div(
                               [
                                 dbc.Tabs(
                                            [
                                              dbc.Tab(label="Loan 1", tab_id="tab-1"),
                                              dbc.Tab(label="Loan 2", tab_id="tab-2"),
                                              dbc.Tab(label="Loan 3", tab_id="tab-3"),
                                              dbc.Tab(label="Loan Portfolio", tab_id="tab-4"),
                                              dbc.Tab(label="Contribution Impact", tab_id="tab-5")
                                            ],
            id="tabs",
            active_tab="tab-1",
        ),
        html.Div(id="content"),
    ]
),
            id="tables",
            is_in=True,
            appear=False,
        ), md=12),

        
        
        
      
    ],
    id="main-container",
    style={"display": "flex", "flex-direction": "column"},
    fluid=True
)





#============================================== Callbacks ==============================================#

# Input information and check values

@app.callback(
    [Output("principal-input", "valid"), Output("principal-input", "invalid")],
    [Input("principal-input", "value")],
)
def check_validity(number):
    if number > 0:
        return True, False
    else:
        return False, True


@app.callback(
    [Output("min-payment-input", "valid"), Output("min-payment-input", "invalid")],
    [Input("min-payment-input", "value")],
)
def check_validity(number):
    if number > 0:
        return True, False
    else:
        return False, True

    
@app.callback(
    [Output("interest-input", "valid"), Output("interest-input", "invalid")],
    [Input("interest-input", "value")],
)
def check_validity(number):
    if number > 0:
        return True, False
    else:
        return False, True


@app.callback(
    [Output("extra-input", "valid"), Output("extra-input", "invalid")],
    [Input("extra-input", "value")],
)
def check_validity(number):
    if number >= 0:
        return True, False
    else:
        return False, True
    
    
# Contribution information and check values

@app.callback(
    [Output("contribution1", "valid"), Output("contribution1", "invalid")],
    [Input("contribution1", "value")],
)
def check_validity(number):
    if number > 0:
        return True, False
    else:
        return False, True

    
@app.callback(
    [Output("contribution2", "valid"), Output("contribution2", "invalid")],
    [Input("contribution2", "value")],
)
def check_validity(number):
    if number > 0:
        return True, False
    else:
        return False, True

    
@app.callback(
    [Output("contribution3", "valid"), Output("contribution3", "invalid")],
    [Input("contribution3", "value")],
)
def check_validity(number):
    if number > 0:
        return True, False
    else:
        return False, True

    
# Store the multiple loan information input
# loan input


@app.callback(
    Output("memory_input","data"),
    [Input('submit-self-state','n_clicks')],
    [State("principal-input","value"),
     State("interest-input", "value"),
     State("min-payment-input", "value"),
     State("extra-input", "value"),
     State("date-picker-single", "date"),
    ])



def loan_information(n_clicks,principal, rate, payment, extra_payment, date_value):
    
    if ((n_clicks > 0) and (n_clicks < 4)):
        num = n_clicks - 1
        loan_all[num][0] = principal
        loan_all[num][1] = rate
        loan_all[num][2] = payment
        loan_all[num][3] = extra_payment
        date_object = date.fromisoformat(date_value)
        date_string = date_object.strftime('%Y-%m-%d')
        loan_all[num][4] = date_string
        
        return loan_all
    
    if n_clicks >= 4:
        num = n_clicks % 4
        loan_all[num][0] = principal
        loan_all[num][1] = rate
        loan_all[num][2] = payment
        loan_all[num][3] = extra_payment
        date_object = date.fromisoformat(date_value)
        date_string = date_object.strftime('%Y-%m-%d')
        loan_all[num][4] = date_string
        
        return loan_all
    

# Store the contribution Input
@app.callback(Output('memory_contributor', 'data'),
            [Input('submit-contributor-state','n_clicks')],
            [State('contribution_loan_number', 'value'),
             State('contribution1_amount', 'value'),
             State('contribution2_amount', 'value'),
             State('contribution3_amount', 'value'),
             State('contribution1_name', 'value'),
             State('contribution2_name', 'value'),
             State('contribution3_name', 'value')])

def contribution_effect_all(n_clicks,loan_number,contribution1,contribution2,contribution3,contributor1,contributor2,contributor3):
    contribution_info = []
    contributions = []
    contributors = []
        
    if n_clicks:
        contribution_info.append(loan_number)
    
        if contribution1:
            contributions.append(contribution1)
        if contribution2:
            contributions.append(contribution2)
        if contribution3:
            contributions.append(contribution3)
    
        if contributor1:
            contributors.append(contributor1)
        if contributor2:
            contributors.append(contributor2)
        if contributor3:
            contributors.append(contributor3)
        
        contribution_info.append(contributions)
        contribution_info.append(contributors)
    
        return contribution_info
    
    
# show and collapse results

@app.callback(
    Output("tables", "is_in"),
    [Input("fade-button", "n_clicks")],
    [State("tables", "is_in")],
)
def toggle_fade(n, is_in):
    if not n:
        # Button has never been clicked
        return False
    return not is_in
    

    
# tabs of tables and graphs

def compute_schedule(principal, rate, payment, extra_payment, date):
    loan = Loan(principal=principal, rate=rate, payment=payment, extra_payment=extra_payment)
    loan.compute_schedule()
    return Helper.print(loan, date)

def compute_aggregate_schedule(length,loan_list,date):
    loans = LoanPortfolio()
    for i in range(length):
        loan = Loan(loan_list[i][0], loan_list[i][1], loan_list[i][2], loan_list[i][3])
        loan.compute_schedule()
        loans.add_loan(loan)
        
    if loans.get_loan_count() == length:
        loans.aggregate()

    return Helper.print(loans,date)

def loan_visual(df):
    return Helper.plot(df)


def loan_contribution_df(principal, rate, payment, extra_payment, contributions):
    loan_impacts = LoanImpacts(principal=principal, rate=rate, payment=payment,
                               extra_payment=extra_payment, contributions=contributions)
    return loan_impacts.compute_impacts()


def contribution_impact_tab(df,len_final_contributions):
    dff = pd.DataFrame(columns = ['Contributor', 'Interest Saved ($)', 'Duration Saved (Month)'])
    
    duration_list = []
    interest_list = []
    
    if len_final_contributions == 1:
        contributions_option = ['1']
        dff['Contributor'] = list(''.join(sub) for i in range(1, len(contributions_option) + 1) for sub in combinations(contributions_option, i))
        interest_list.append(df.iloc[1,1] - df.iloc[0,1])
        duration_list.append(df.iloc[1,2] - df.iloc[0,2])
    
    if len_final_contributions == 2:
        contributions_option = ['1','2']
        dff['Contributor'] = list(''.join(sub) for i in range(1, len(contributions_option) + 1) for sub in combinations(contributions_option, i))
        interest_list.append(df.iloc[1,1] - df.iloc[3,1])
        interest_list.append(df.iloc[1,1] - df.iloc[2,1])
        interest_list.append(df.iloc[1,1] - df.iloc[0,1])
        
        duration_list.append(df.iloc[1,2] - df.iloc[3,2])
        duration_list.append(df.iloc[1,2] - df.iloc[2,2])
        duration_list.append(df.iloc[1,2] - df.iloc[0,2])
    
    if len_final_contributions == 3:
        contributions_option = ['1','2','3']
        dff['Contributor'] = list(''.join(sub) for i in range(1, len(contributions_option) + 1) for sub in combinations(contributions_option, i))
        interest_list.append(df.iloc[2,1] - df.iloc[0,1])
        interest_list.append(df.iloc[3,1] - df.iloc[0,1])
        interest_list.append(df.iloc[4,1] - df.iloc[0,1])
        interest_list.append(df.iloc[1,1] - df.iloc[4,1])
        interest_list.append(df.iloc[1,1] - df.iloc[3,1])
        interest_list.append(df.iloc[1,1] - df.iloc[2,1])
        interest_list.append(df.iloc[1,1] - df.iloc[0,1])
        
        duration_list.append(df.iloc[2,2] - df.iloc[0,2])
        duration_list.append(df.iloc[3,2] - df.iloc[0,2])
        duration_list.append(df.iloc[4,2] - df.iloc[0,2])
        duration_list.append(df.iloc[1,2] - df.iloc[4,2])
        duration_list.append(df.iloc[1,2] - df.iloc[3,2])
        duration_list.append(df.iloc[1,2] - df.iloc[2,2])
        duration_list.append(df.iloc[1,2] - df.iloc[0,2])
    
    dff['Interest Saved ($)'] = interest_list
    dff['Duration Saved (Month)'] = duration_list
    
    dff = dff.sort_values(by=['Interest Saved ($)','Duration Saved (Month)'], ascending=False)
    
    return dff

def format(x):
    return "${:.2f}".format(x)


def contribution_plot(variable,df):
    fig = px.pie(df, values=variable, names='Names',\
             color_discrete_sequence=px.colors.qualitative.Pastel)
    fig.update_traces(textinfo='percent+label')
    return fig

@app.callback(Output("content", "children"), 
              [Input("tabs", "active_tab"),
              Input("memory_input","data"),
              Input("memory_contributor","data")])
def switch_tab(at,loan_all,contribution_info):
    if at == "tab-1":
        df_loan1 = compute_schedule(loan_all[0][0], loan_all[0][1], loan_all[0][2], loan_all[0][3],loan_all[0][4])
        date_loan1 = datetime.strptime(df_loan1.iloc[-1,0], '%b-%y').strftime('%b %d,%Y')
        graph_loan1 = loan_visual(df_loan1)
        return html.Div(
                          [
                           dbc.Row(
                                     [
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P("Estimated Payoff Date",
                                                    style={
                                                            'textAlign': 'right',
                                                            'fontSize': 20
                                                            })),
                                      ]
                                   ),
                              
                           dbc.Row(
                                     [
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P(date_loan1,
                                                      style={
                                                      'textAlign': 'right',
                                                      'fontSize': 30
                                                     })),
                                      ]
                                   ),
                        
                           dbc.Row(dbc.Col(html.H5('Amortization Schedule',
                                                    style={
                                                            'textAlign': 'left',
                                                            'fontSize': 25
                                                            }))),
                           
                           dbc.Row(dbc.Table.from_dataframe(df_loan1)),
                              
                           dbc.Row(dcc.Graph(
                                      id='loan 1 graph',
                                      figure = graph_loan1
                                     )),

                            ]
                         )
   
    
    if at == "tab-2":
        if loan_all[1][0] != 0:
            df_loan2 = compute_schedule(loan_all[1][0], loan_all[1][1], loan_all[1][2], loan_all[1][3],loan_all[1][4])
            date_loan2 = datetime.strptime(df_loan2.iloc[-1,0], '%b-%y').strftime('%b %d,%Y')
            graph_loan2 = loan_visual(df_loan2)
            return html.Div(
                          [
                           dbc.Row(
                                     [
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P("Estimated Payoff Date",
                                                    style={
                                                            'textAlign': 'right',
                                                            'fontSize': 20
                                                            })),
                                      ]
                                   ),
                              
                           dbc.Row(
                                     [
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P(date_loan2,
                                                    style={
                                                            'textAlign': 'right',
                                                            'fontSize': 30
                                                            })),
                                      ]
                                   ),
                        
                           dbc.Row(dbc.Col(html.H5('Amortization Schedule',
                                                    style={
                                                            'textAlign': 'left',
                                                            'fontSize': 25
                                                            }))),
                           
                           dbc.Row(dbc.Table.from_dataframe(df_loan2)),
                              
                           dbc.Row(dcc.Graph(
                                      id='loan 2 graph',
                                      figure = graph_loan2
                                     )),

                            ]
                         )
        else:        
            return html.Div(html.P('No Loan Information'))
    
    if at == "tab-3":
        if loan_all[2][0] != 0:
            df_loan3 = compute_schedule(loan_all[2][0], loan_all[2][1], loan_all[2][2], loan_all[2][3],loan_all[2][4])
            date_loan3 = datetime.strptime(df_loan3.iloc[-1,0], '%b-%y').strftime('%b %d,%Y')
            graph_loan3 = loan_visual(df_loan3)
            return html.Div(
                          [
                           dbc.Row(
                                     [
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P("Estimated Payoff Date",
                                                    style={
                                                            'textAlign': 'right',
                                                            'fontSize': 20
                                                            })),
                                      ]
                                   ),
                              
                           dbc.Row(
                                     [
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P(date_loan3,
                                                    style={
                                                            'textAlign': 'right',
                                                            'fontSize': 30
                                                            })),
                                      ]
                                   ),
                        
                           dbc.Row(dbc.Col(html.H5('Amortization Schedule',
                                                    style={
                                                            'textAlign': 'left',
                                                            'fontSize': 25
                                                            }))),
                           
                           dbc.Row(dbc.Table.from_dataframe(df_loan3)),
                              
                           dbc.Row(dcc.Graph(
                                      id='loan 3 graph',
                                      figure = graph_loan3
                                     )),

                            ]
                         )
        else:        
            return html.Div(html.P('No Loan Information'))
    
    if at == "tab-4":
        loans = compute_aggregate_schedule(len(loan_all), loan_all,loan_all[0][4])
        date_loans = datetime.strptime(loans.iloc[-1,0], '%b-%y').strftime('%b %d,%Y')
        graph_loans = loan_visual(loans)
        return html.Div(
                          [
                           dbc.Row(
                                     [
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P("Estimated Payoff Date",
                                                    style={
                                                            'textAlign': 'right',
                                                            'fontSize': 20
                                                            })),
                                      ]
                                   ),
                              
                           dbc.Row(
                                     [
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P(" ")),
                                      dbc.Col(html.P(date_loans,
                                                    style={
                                                            'textAlign': 'right',
                                                            'fontSize': 30
                                                            })),
                                      ]
                                   ),
                        
                           dbc.Row(dbc.Col(html.H5('Amortization Schedule',
                                                    style={
                                                            'textAlign': 'left',
                                                            'fontSize': 25
                                                            }))),
                           
                           dbc.Row(dbc.Table.from_dataframe(loans)),
                              
                           dbc.Row(dcc.Graph(
                                      id='loan all graph',
                                      figure = graph_loans
                                     )),

                            ]
                         )

        


    if at == "tab-5":
        i = contribution_info[0] - 1
        contribution_df = loan_contribution_df(loan_all[i][0], loan_all[i][1], loan_all[i][2], loan_all[i][3],contribution_info[1])
        contribution_final_df = contribution_impact_tab(contribution_df,len(contribution_info[1]))     
        
        contribution_final_table = contribution_final_df.copy()
        contribution_final_table['Interest Saved ($)'] =contribution_final_table['Interest Saved ($)'].apply(format)
        
        
        contributors = contribution_info[2]

        def assign_label(i):
            if i =='1':
                return contributors[0]
            if i =='2':
                return contributors[1]
            if i == '3':
                return contributors[2]

        contribution_final_df['Names'] = contribution_final_df['Contributor'].apply(assign_label)
        contribution_plot_df = contribution_final_df[(contribution_final_df['Contributor']=='1')|(contribution_final_df['Contributor']=='2')|(contribution_final_df['Contributor']=='3')]
        interest_fig = contribution_plot('Interest Saved ($)',contribution_plot_df)
        duration_fig = contribution_plot('Duration Saved (Month)',contribution_plot_df)
        
        # plot of interest

        first_card = dbc.Card(
                              [
                               dbc.CardBody(
                                            [
                                              html.P('Saved Interest Distribution',
                                              style={
                                                     'textAlign': 'center',
                                                     'fontSize': 25
                                                     }),
                                             
                                              html.Div(dbc.Row(dcc.Graph(
                                                                         figure = interest_fig
                                                                        ))
                                                      )
          
                                              ]
                                             )
                              ]
                                )

        # plot of duration

        second_card = dbc.Card(
                              [
                               dbc.CardBody(
                                            [
                                              html.P('Saved Duration Distribution',
                                              style={
                                                     'textAlign': 'center',
                                                     'fontSize': 25
                                                     }),
                                             
                                              html.Div(dbc.Row(dcc.Graph(
                                                                         figure = duration_fig
                                                                        ))
                                                      )
          
                                              ]
                                             )
                              ]
                                )


              
        return html.Div(
                          [
                          dbc.Row(html.H5(html.Div(id='display-contribution-title'),
                                                   style={
                                                          'textAlign': 'left',
                                                          'fontSize': 35
                                                          })),
                           dbc.Row([html.Hr(),
                                    html.Br()]),
                           dbc.Row(html.H6('Table of Different Combinations of Contributions and Impacts')),
                           dbc.Row(html.Br()),
                           dbc.Row(dbc.Table.from_dataframe(contribution_final_table)),
                           dbc.Row(html.Br()),
                           dbc.Row(html.H6('Plots of the Distribution of Interest/Duration Saved')),
                           dbc.Row(html.Br()),
                           dbc.Row([dbc.Col(first_card, width=6), dbc.Col(second_card, width=6)])  
                           

                            ]
                        )


# Overview of monthly payment, total principal paid, and total interest paid:

@app.callback(Output('monthly payment overview', 'children'),
            [Input("memory_input","data")])

def display_monthly_payment(loan_all):
    df = compute_aggregate_schedule(len(loan_all), loan_all,loan_all[0][4])
    min_pay = df['Payment'].str.extract(r'(\d+\.\d+|\d+)', expand=False).astype(float).round(2)[0]
    extra_pay = df['Extra Payment'].str.extract(r'(\d+\.\d+|\d+)', expand=False).astype(float).round(2)[0]
    month_pay = min_pay + extra_pay 
    return '${:.2f}'.format(month_pay)

@app.callback(Output('Total Principal Paid', 'children'),
            [Input("memory_input","data")])

def display_principal_payment(loan_all):
    df = compute_aggregate_schedule(len(loan_all), loan_all,loan_all[0][4])
    principal_pay = df['Applied Principal'].str.extract(r'(\d+\.\d+|\d+)', expand=False).astype(float).round(2).sum().round(2)
    return 'Total Principal Paid: ${:.2f}'.format(principal_pay)

@app.callback(Output('Total Interest Paid', 'children'),
            [Input("memory_input","data")])

def display_interest_payment(loan_all):
    df = compute_aggregate_schedule(len(loan_all), loan_all,loan_all[0][4])
    interest_pay = df['Applied Interest'].str.extract(r'(\d+\.\d+|\d+)', expand=False).astype(float).round(2).sum().round(2)
    return 'Total Interest Paid: ${:.2f}'.format(interest_pay)


# Contribution Title Display:
@app.callback(Output('display-contribution-title', 'children'),
            [Input("memory_contributor","data")])

def display_contribution_title(contribution_info):
    num = contribution_info[0]
    return u'The Contribution Impact of Loan {}:'.format(num)

   

if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False,dev_tools_ui=False,dev_tools_props_check=False,\
                   port=8060, host='127.0.0.1')  

Dash is running on http://127.0.0.1:8060/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
